# Web Search
- LLM에게 웹 검색을 쥐어주자
- tavily 사용 (1000건까지 free)

In [1]:
%pip install -qU langchain-tavily

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from dotenv import load_dotenv
from pprint import pprint
load_dotenv()

True

In [3]:
from langchain_tavily import TavilySearch

search_tool = TavilySearch(
    max_results=5, # 몇 개 찾아올건지
    topic="general", # news, finance이라고 설정 가능
    search_depth="basic", # advanced는 deep search
    
    # include_answer=False, # 내 질문 포함 할말
    # include_raw_content=False, # html 포함?
    # include_images=False,
    # include_image_descriptions=False,
    # time_range="day", # 검색 데이터의 최신성
    # include_domains=None, 
    # exclude_domains=None
)

In [ ]:
# invoke가 있다는 건 chain에 포함할 수 있다!!
search_tool.invoke({'query': '랭체인 tavily 어떻게 쓰는거야?'})

{'query': '랭체인 tavily 어떻게 쓰는거야?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://wikidocs.net/261568',
   'title': '4-2-1. 웹 검색 도구 활용 - 랭체인(LangChain) 입문부터 응용까지',
   'content': "기본 LLM 체인 (Prompt + LLM)) LangChain 모델 유형) 유사도 기반 검색 (Similarity search)) 유사도 기반 검색 (Similarity search) (Copy)) 올라마(Ollama) 모델에 랭체인(LangChain) 적용) Groq API 모델을 랭체인(LangChain) 적용) 웹 검색 도구 활용) 웹 검색 도구 활용 웹 검색 도구 활용 # LangChain의 웹 검색 도구 사용 LangChain의 웹 검색 도구인 Tavily API를 사용하여 실시간 금융 정보를 검색하고 분석에 활용하는 예제를 살펴보겠습니다. Tavily API를 사용하여 LangChain의 웹 검색 기능을 활용하기 위해서는 API 인증키가 필요합니다. 이 예제에서는 2025년 애플의 주가 전망에 대한 분석을 요청합니다. 검색 도구 초기화 검색 결과는 `search_results` 변수에 저장됩니다. {'url': 'https://therapycenter.kr/contents/애플-주식-분석-2025년-8월-전망과-투자-전략/', 'content': '특히 2025년 8월을 기준으로 애플 주식의 향후 전망은 기술 혁신, 제품 출시, 그리고 글로벌 경제 동향에 따라 중요한 변화를 맞이할 수 있습니다. 이 정보를 바탕으로 애플의 2025년 주가 전망에 대한 분석을 수행할 수 있습니다.",
   'score': 0.71377134,
   'raw_content': None},
  {'url': 'https://steemit.com/krdev/@anpigon/2024

In [20]:
from langchain.prompts import PromptTemplate # 프롬포트 담당
from langchain_openai import ChatOpenAI # AI 담당
from langchain_core.output_parsers import StrOutputParser # 파서 담당
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

llm = ChatOpenAI(model='gpt-4.1-nano')

prompt = PromptTemplate(
    input_variables=['query', 'search_results'],

    template='''넌 웹 검색결과를 요약해서 사용자 질문에 맞게 좋은 답을 해주는 챗봇이야.

    질문: {query}
    검색결과: {search_results}
    질문에 맞는 답을 검색결과 바탕으로 간단하게 알려줘
    '''
)


chain = (
    {
        'query': RunnablePassthrough(), # 사용자 입력 그대로 이 자리에 들어옴
        'search_results': search_tool | RunnableLambda(
            lambda x: '\n'.join(
                f'-{r['title']}: {r['content']}' for r in x['results'] # content말고 url이라고만 하면 내용 제대로 안나옴
        )
        )
    }
    | prompt | llm | StrOutputParser()
)

print(chain.invoke('강남 신세계 디저트 추천해줘'))

강남 신세계백화점 내 디저트 추천으로는, 전통 한식 디저트와 프랑스식 디저트가 모두 있습니다. 추천하는 곳은 '스위트파크'로, 다양한 디저트와 빵, 마카롱, 마들렌 등 맛있는 디저트를 즐기실 수 있으며, 분위기도 좋아요. 또, 일본 오모테산도 출신의 디저트 브랜드 '가리게트'도 작은 규모지만 섬세한 디저트를 맛볼 수 있습니다.
